# Intent Classification 

## Clinc 150 Dataset from UCI
[An evaluation datset for intent classification and out-of-scope prediction.](https://archive.ics.uci.edu/ml/datasets/CLINC150)

**Citation**
* Stefan Larson, Anish Mahendran, Joseph J. Peper, Christopher Clarke, Andrew Lee, Parker Hill, Jonathan K. Kummerfeld, Kevin Leach, Michael A. Laurenzano, Lingjia Tang, and Jason Mars. 2019. An evaluation dataset for intent classification and out-of-scope prediction. In Proceedings of EMNLP-IJCNLP

#### import dependencies

In [1]:
import os
import numpy as np
from sklearn.svm import SVC